### Imports

In [1]:
# Packages
import tensorflow as tf

#### Build model

In [2]:
model = tf.keras.Sequential()

In [3]:
model.add(tf.keras.Input(shape=8, name="inputLayer"))
model.add(tf.keras.layers.Dense(512*4))
#model.add(tf.keras.layers.Conv1D(40, 10, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation = tf.nn.softmax))

In [4]:
model.compile(loss=tf.keras.losses.MeanAbsoluteError(), 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=tf.keras.metrics.Accuracy())

In [ ]:
model.summary()

Train

In [5]:
model.fit(train_dataset_tf, epochs=10)

NameError: name 'train_dataset_tf' is not defined

Eva